In [ ]:
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

import pandas as pd
import numpy as np
from scipy.stats import percentileofscore

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from resources.plotly_theme import nfl_template
from resources.heat_map import heat_map
from resources.get_nfl_data import get_pbp_data, get_team_info
from resources.team_stats import get_team_stats
from resources.player_stats import get_player_stats

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Parameters '''

WEEK = 10
AWAY_TEAM = 'ATL'
HOME_TEAM = 'IND'

EXPORT = False

# Process

In [ ]:
''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025])

player_info = nfl.import_players()

## Data ##
league_data = pbp_data.loc[(pbp_data['week'] <= WEEK), :].copy()

game_data = pbp_data.loc[(pbp_data['week'] == WEEK) & 
                         (pbp_data['home_team'] == HOME_TEAM), :].copy()
run_data = game_data.loc[game_data['rush'] == 1, :].copy()
pass_data = game_data.loc[game_data['pass'] == 1, :].copy()

## Offense ##
league_offense = get_team_stats(league_data, unit='offense')
team_offense = get_team_stats(game_data, unit='offense')

## Team Defense ##
team_defense = get_team_stats(game_data, unit='defense')


print(game_data.shape)
print(team_offense.to_string())
print(team_defense.to_string())
print(player_info.head(2).to_string())


In [ ]:
''' Percentiles '''

cols = ['EPA / Play', 'Pass EPA / Play', 'Rush EPA / Play', 'Success Rate', 'Pass Success Rate', 'Rush Success Rate']

data = team_offense.melt(
    ignore_index=False,
    value_vars=cols,
    var_name='Metric',
    value_name='Value'
)
data['Percentile'] = 0.0

for col in cols:
    league_vals = league_offense[col].to_numpy()

    for team in team_offense.index.tolist():
        conditions = (data.index == team) & (data['Metric'] == col)
        team_val = data.loc[conditions, 'Value'].values[0]
        percentile = percentileofscore(league_vals, team_val) / 100
        data.loc[conditions, 'Percentile'] = percentile

data = data.merge(team_data['team_logo_espn'], left_index=True, right_index=True)

print(data.head().to_string())

In [ ]:

# Data
x = data['Percentile'].tolist()
y = data['Metric'].tolist()
text = data['Value'].tolist()
logos = data['team_logo_espn'].tolist()

# y_order = ['1st & 10']
# for down in ['2nd', '3rd', '4th']:
#     for distance in ['Short', 'Medium', 'Long']:
#         y_order.append(f'{down} & {distance}')

# Plot
dot_plot = px.scatter(
    x=x,
    y=y,
)

# Figure
fig = go.Figure()

for trace in dot_plot.data:
    fig.add_trace(trace)

# Logos
for i in range(len(logos)):
    fig.add_layout_image(
        source=logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=y[i], # Y-coordinate of the image's center
        sizex=0.75,   # Width of the image in data units
        sizey=0.75,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )

fig.update_traces(marker=dict(opacity=0))

# x_ticks = [abs(xi) for xi in x]
fig.update_xaxes(
    title=f'More Run Heavy <--  % Pass  --> More Pass Heavy',
    tickformat='.0%',
    range=[-0.05, 1.05],
    dtick=.1,
    linecolor='#f0f0f0', mirror=True
)
fig.update_yaxes(
    # type='category',
    # categoryorder="array", 
    # categoryarray=y_order,
    autorange='reversed',
    linecolor='#f0f0f0',mirror=True,
    showgrid=False,
)
fig.update_layout(
    template='nfl_template',
    title=f'Pass / Rush Tendencies by Down & Distance',#<br><sup>Week {PREVIEW_WEEK}: {AWAY_TEAM} vs. {HOME_TEAM}</sup>',
    margin=dict(t=60,l=100, b=75),
)

# Credits
fig.add_annotation(
    text=f'"Normal" game state: Qtrs 1-3, score within 14 pts<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.175, 
    x=1,
    align='right'
)

fig.show()


In [ ]:
''' Team Rushing '''

team_rushing = run_data.groupby(['posteam', 'game_half']).aggregate(
    Plays=('rush', 'sum'),
    Attempts=('rush_attempt', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum')
)
totals = team_rushing.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
team_rushing = pd.concat([team_rushing, totals]).sort_index()

team_rushing['Yds / Att'] = round(team_rushing['Yards'] / team_rushing['Attempts'], 2)
team_rushing['Success Rate'] = round((team_rushing['Successes'] / team_rushing['Attempts']) * 100, 2)
team_rushing['1D Rate'] = round((team_rushing['FirstDowns'] / team_rushing['Attempts']) * 100, 2)
team_rushing['TD Rate'] = round((team_rushing['TDs'] / team_rushing['Attempts']) * 100, 2)

print(team_rushing.to_string())


In [ ]:
''' Player Rushing '''

by_rusher = run_data.groupby(['posteam', 'rusher']).aggregate(
    Plays=('rush', 'sum'),
    Attempts=('rush_attempt', 'sum'),
    Yards=('rushing_yards', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
    WPA=('wpa', 'sum'),
).sort_values(by=['posteam', 'Attempts'], ascending=False)

by_rusher['Yds / Att'] = round(by_rusher['Yards'] / by_rusher['Attempts'], 2)
by_rusher['Success Rate'] = round((by_rusher['Successes'] / by_rusher['Attempts']) * 100, 2)
by_rusher['EPA / Play'] = round((by_rusher['EPA'] / by_rusher['Plays']), 2)
by_rusher['1D Rate'] = round((by_rusher['FirstDowns'] / by_rusher['Attempts']) * 100, 2)
by_rusher['TD Rate'] = round((by_rusher['TDs'] / by_rusher['Attempts']) * 100, 2)

print(by_rusher.to_string())

In [ ]:
colors = px.colors.sample_colorscale('PRGn', samplepoints=[0,1])
print(colors)

In [ ]:
''' Rush SR by Down / Distance '''

MIN_ATTEMPTS = 1

## Group by down/distance
sr_slice = run_data.copy()
sr_slice.loc[sr_slice['Down & Distance'].str.contains('1st'), 'Down & Distance'] = '1st Down'

rusher_carries = sr_slice.groupby(['rusher', 'Down & Distance']).aggregate(
    Attempts=('rush_attempt', 'sum'),
    Successes=('success', 'sum'),
    player_id=('rusher_player_id', 'first'),
).reset_index()
rusher_carries['Success Rate'] = rusher_carries['Successes'] / rusher_carries['Attempts']

## Pivot
DOWNS = ['1st Down', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS, ['Successes', 'Attempts', 'Success Rate']])

piv = rusher_carries.loc[(rusher_carries['Down & Distance'] != ''),:].pivot(
        index='rusher',
        columns=['Down & Distance'],
        values=['Attempts', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('rusher')\
    .reindex(columns=col_sort)

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total attempts
sl['Attempts'] = sl.index.map(by_rusher.groupby('rusher')['Attempts'].sum())
sl = sl.loc[sl['Attempts'] >= MIN_ATTEMPTS, :].sort_values(by='Attempts', ascending=False)

# print(sl.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    for c in piv.columns.get_level_values(0).unique():
        attempts = piv.loc[r, (c, 'Attempts')].astype(float)
        successes = piv.loc[r, (c, 'Successes')].astype(float)
        success_rate = piv.loc[r, (c, 'Success Rate')].astype(float) * 100
        if pd.isna(attempts):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{success_rate:,.1f}%<br>({successes:,.0f} / {attempts:,.0f})"


## Heatmap ##
player_ids = rusher_carries.reset_index().groupby('rusher')['player_id'].first()
sl['player_id'] = sl.index.map(player_ids)
sl['headshot'] = sl['player_id'].map(player_info.set_index('gsis_id')['headshot'])

x = [down.replace(' ', '<br>') for down in DOWNS]
y = sl.index
z = sl[DOWNS].values.tolist()
headshots = sl['headshot'].tolist()
n_players = len(headshots)

# Create
fig = heat_map(
    x=x,
    y=y,
    z=z,
    text=annot_df.values.tolist(),
    title=f'<b>Rushing Success Rate by Down & Distance</b><br><sup>Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}; min {MIN_ATTEMPTS} attempts</sup>',
    color_name='Success Rate',
)
fig.update_traces(
    colorbar=dict(x=1)
)

# Headshots
for i in range(n_players):
    response = requests.get(headshots[i])
    headshot = Image.open(BytesIO(response.content))

    hs_size = (1/n_players) * 0.9
    y = (1 - ((1/n_players)/2)) - ((1/n_players)*i)
    fig.add_layout_image(
        source=headshot,
        xref='paper', 
        yref='paper',
        x=-.125,
        y=y,
        sizex=hs_size,
        sizey=hs_size,
        xanchor='center',
        yanchor='middle',
    )

# Format
fig.update_yaxes(
    showgrid=False,
    tickfont=dict(size=10),
    linecolor='#f0f0f0', mirror=True
)
fig.update_xaxes(
    linecolor='#f0f0f0', mirror=True
)
fig.update_layout(
    margin=dict(t=100, l=125, r=0, b=40),
    margin_pad=5,
    width=900,
    height=500
)
fig.update_annotations(y=-.075)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f"Rushing SR by Down & Distance - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png", height=500, width=900, scale=6)

## Passing

In [ ]:
''' Team Passing '''
# NOTE - Team passing includes sacks, both attempts and yards

team_passing = pass_data.groupby(['posteam', 'game_half']).aggregate(
    Plays=('pass', 'sum'),
    Attempts=('pass_attempt', 'sum'),
    Completions=('complete_pass', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    INTs=('interception', 'sum'),
    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[pass_data['sack'] == 1].sum()),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
)
totals = team_passing.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
team_passing = pd.concat([team_passing, totals]).sort_index()
team_passing['Attempts'] = team_passing['Attempts'] - team_passing['Sacks']

team_passing['Yds / Att'] = round(team_passing['Yards'] / team_passing['Attempts'], 2)
team_passing['Success Rate'] = round((team_passing['Successes'] / team_passing['Plays']) * 100, 2)
team_passing['EPA / Play'] = round((team_passing['EPA'] / team_passing['Plays']), 2)
team_passing['1D Rate'] = round((team_passing['FirstDowns'] / team_passing['Attempts']) * 100, 2)
team_passing['TD Rate'] = round((team_passing['TDs'] / team_passing['Attempts']) * 100, 2)

print(team_passing.to_string())

In [ ]:
''' Passing - QBs '''


by_passer = pass_data.groupby(['posteam', 'passer']).aggregate(
    Plays=('pass', 'sum'),
    Attempts=('pass_attempt', 'sum'),
    Completions=('complete_pass', 'sum'),
    Yards=('passing_yards', 'sum'),
    TDs=('touchdown', 'sum'),
    INTs=('interception', 'sum'),
    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[pass_data['sack'] == 1].sum()),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
    WPA=('wpa', 'sum'),
)
by_passer['Attempts'] = by_passer['Attempts'] - by_passer['Sacks']

by_passer['Yds / Att'] = round(by_passer['Yards'] / by_passer['Attempts'], 2)
by_passer['Success Rate'] = round((by_passer['Successes'] / by_passer['Plays']) * 100, 2)
by_passer['EPA / Play'] = round((by_passer['EPA'] / by_passer['Plays']), 2)
by_passer['1D Rate'] = round((by_passer['FirstDowns'] / by_passer['Attempts']) * 100, 2)
by_passer['TD Rate'] = round((by_passer['TDs'] / by_passer['Attempts']) * 100, 2)

print(by_passer.to_string())

In [ ]:
''' Receiving '''
# NOTE - 

by_receiver = pass_data.groupby(['posteam', 'receiver']).aggregate(
    Plays=('pass', 'sum'),
    Targets=('pass_attempt', 'sum'),
    Receptions=('complete_pass', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
    WPA=('wpa', 'sum'),
).sort_values(by=['posteam', 'Targets'], ascending=False)

by_receiver['Yds / Rec'] = round(by_receiver['Yards'] / by_receiver['Receptions'], 2)
by_receiver['Success Rate'] = round((by_receiver['Successes'] / by_receiver['Plays']) * 100, 2)
by_receiver['EPA / Play'] = round((by_receiver['EPA'] / by_receiver['Plays']), 2)
by_receiver['1D Rate'] = round((by_receiver['FirstDowns'] / by_receiver['Plays']) * 100, 2)
by_receiver['TD Rate'] = round((by_receiver['TDs'] / by_receiver['Plays']) * 100, 2)

print(by_receiver.to_string())

In [ ]:
EXPORT = False

In [ ]:
''' Receiving by Down / Distance '''

DOWN_COL = 'Down & Distance'
MIN_PLAYS = 2

## Group by down/distance
sr_slice = pass_data.copy()
sr_slice.loc[sr_slice['Down & Distance'].str.contains('1st'), 'Down & Distance'] = '1st Down'

receiver_sr_by_down = sr_slice.groupby(['receiver', DOWN_COL]).aggregate(
    Plays=('pass', 'sum'),
    Successes=('success', 'sum'),
    player_id=('receiver_player_id', 'first'),
).reset_index()
receiver_sr_by_down['Success Rate'] = receiver_sr_by_down['Successes'] / receiver_sr_by_down['Plays']
# receiver_sr_by_down = receiver_sr_by_down.loc[receiver_sr_by_down['Plays'] >= 1, :]

# print(receiver_sr_by_down.to_string())

## Pivot
DOWNS = ['1st Down', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS, ['Successes', 'Plays', 'Success Rate']])

piv = receiver_sr_by_down.loc[(receiver_sr_by_down['Down & Distance'] != ''),:].pivot(
        index='receiver',
        columns=['Down & Distance'],
        values=['Plays', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('receiver')\
    .reindex(columns=col_sort)

# print(piv.to_string())

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total attempts
sl['Plays'] = sl.index.map(receiver_sr_by_down.groupby('receiver')['Plays'].sum())
sl = sl.loc[sl['Plays'] >= MIN_PLAYS, :].sort_values(by='Plays', ascending=False)

# print(sl.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    for c in piv.columns.get_level_values(0).unique():
        plays = piv.loc[r, (c, 'Plays')].astype(float)
        successes = piv.loc[r, (c, 'Successes')].astype(float)
        success_rate = piv.loc[r, (c, 'Success Rate')].astype(float) * 100
        if pd.isna(plays):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{success_rate:,.1f}%<br>({successes:,.0f} / {plays:,.0f})"


## Heatmap ##
player_ids = receiver_sr_by_down.reset_index().groupby('receiver')['player_id'].first()
sl['player_id'] = sl.index.map(player_ids)
sl['headshot'] = sl['player_id'].map(player_info.set_index('gsis_id')['headshot'])

x = [down.replace(' ', '<br>') for down in DOWNS]
y = sl.index
z = sl[DOWNS].values.tolist()
headshots = sl['headshot'].tolist()
n_players = len(headshots)

# Create
fig = heat_map(
    x=x,
    y=y,
    z=z,
    text=annot_df.values.tolist(),
    title=f'<b>Receiving Success Rate by Down & Distance</b><br><sup>Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}; min {MIN_PLAYS} plays</sup>',
    color_name='Success Rate',
)
fig.update_traces(
    colorbar=dict(x=1)
)

# Headshots
for i in range(n_players):
    response = requests.get(headshots[i])
    headshot = Image.open(BytesIO(response.content))

    hs_size = (1/n_players) * 0.9
    y = (1 - ((1/n_players)/2)) - ((1/n_players)*i)
    fig.add_layout_image(
        source=headshot,
        xref='paper', 
        yref='paper',
        x=-.125,
        y=y,
        sizex=hs_size,
        sizey=hs_size,
        xanchor='center',
        yanchor='middle',
    )

# Format
fig.update_yaxes(
    showgrid=False,
    tickfont=dict(size=10),
    linecolor='#f0f0f0', mirror=True
)
fig.update_xaxes(
    linecolor='#f0f0f0', mirror=True
)
fig.update_layout(
    margin=dict(t=100, l=125, r=0, b=40),
    margin_pad=5,
    width=900,
    height=500
)
fig.update_annotations(y=-.075)

fig.show()

if EXPORT: pio.write_image(fig, f"Receiving SR by Down & Distance - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png", height=500, width=900, scale=6)

# Players

In [ ]:

passer_sl = by_passer[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'passer': 'player'})
passer_sl.columns = ['Passing ' + col for col in passer_sl.columns]

receiver_sl = by_receiver[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'receiver': 'player'})
receiver_sl.columns = ['Receiving ' + col for col in receiver_sl.columns]

rusher_sl = by_rusher[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'rusher': 'player'})
rusher_sl.columns = ['Rushing ' + col for col in rusher_sl.columns]

## Combine ##
player_epa = passer_sl.merge(receiver_sl, left_index=True, right_index=True, how='outer')
player_epa = player_epa.merge(rusher_sl, left_index=True, right_index=True, how='outer')
player_epa = player_epa.fillna(0)

player_epa['Total Plays'] = player_epa['Passing Plays'] + player_epa['Receiving Plays'] + player_epa['Rushing Plays']
player_epa['Total EPA'] = player_epa['Passing EPA'] + player_epa['Receiving EPA'] + player_epa['Rushing EPA']
player_epa = player_epa.sort_values(by='Total EPA', ascending=False)

player_epa['team_logo_espn'] = player_epa.index.get_level_values(0).map(team_data['team_logo_espn'])
player_epa['team_color'] = player_epa.index.get_level_values(0).map(team_data['team_color'])
# player_epa['headshot'] = player_epa.index.get_level_values(1).map(player_info['headshot'])

print(player_epa.to_string())

In [ ]:
player_stats = get_player_stats(pbp_data=game_data)
player_stats = player_stats.sort_values(by='Total EPA', ascending=False)

print(player_stats.to_string())

In [ ]:
''' Player Contributions - Offense '''

## Data ##
x=player_stats['Total EPA'].tolist()
y=player_stats.index.get_level_values(1)
text=player_stats['Total Plays'].tolist()
logos = player_stats['team_logo_espn'].tolist()
colors = player_stats['team_color'].tolist()

n_players = len(x)

## Bar ##
bar = px.bar(
    x=x,
    y=y,
    text=text
)

fig = go.Figure()

for trace in bar.data:
    fig.add_trace(trace)

# Logos

for i in range(len(logos)):
    fig.add_layout_image(
        x=-0.1,
        y=(1-((1/n_players)*.5))-((1/(n_players))*i),
        sizex=(1/n_players)*.9,
        sizey=(1/n_players)*.9,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logos[i],
    )

# Format
fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=1.5, color='black')))
fig.update_yaxes(
    autorange='reversed',
    linecolor='#f0f0f0', mirror=True
)
fig.update_xaxes(
    title='Total EPA',
    tickformat='.0f',
    linecolor='#f0f0f0', mirror=True
)
fig.update_layout(
    template='nfl_template',
    title=dict(text=f'<b>Offensive Player Contributions</b><br><sup>How each player contributed to the game\'s expected margin in Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>'),
    height=500, width=900,
    margin=dict(t=50, r=25, b=65, l=100)
)

# Credits
fig.add_annotation(
    text=f'Total EPA on offensive plays, passing + receiving + rushing; bar text is total plays<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.15, 
    x=1,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Offensive Player Contributions - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png', scale=6, width=900, height=500)

In [ ]:
''' EPA Box Score '''

## Bar ##
teams = [AWAY_TEAM, HOME_TEAM]
wordmarks = [team_data.loc[team_data.index == AWAY_TEAM, 'team_wordmark'].values[0], team_data.loc[team_data.index == HOME_TEAM, 'team_wordmark'].values[0]]
units = ['Passing', 'Receiving', 'Rushing']

bar_charts = []
x_ranges = []
for unit in units:
    plays_col = f'{unit} Plays'
    epa_col = f'{unit} EPA'

    unit_sl = player_stats.loc[player_stats[plays_col] > 0, :].copy()
    unit_sl = unit_sl.sort_values(by=plays_col, ascending=False)

    r = abs(unit_sl[epa_col].max() - unit_sl[epa_col].min())
    epa_range = [unit_sl[epa_col].min() - (r*.2), unit_sl[epa_col].max() + (r*.2)]
    if epa_range[0] > 0:
        epa_range[0] = 0 - (r*.2)
    if epa_range[1] < 0:
        epa_range[1] = 0 + (r*.2)

    x_ranges.append(epa_range)

    for team in teams:
        team_sl = unit_sl.loc[unit_sl.index.get_level_values(0) == team, :]

        x = team_sl[epa_col].tolist()
        y = team_sl.index.get_level_values(1)
        text = team_sl[plays_col].tolist()
        colors = team_sl['team_color'].tolist()

        bar = px.bar(
            x=x,
            y=y,
            text=text,
            range_x=epa_range,
            color_discrete_sequence=colors,
            opacity=0.7
        )

        bar_charts.append(bar)


## Create Figure ##
N_ROWS = 3
N_COLS = 2

H_SPACING = .25/N_COLS
V_SPACING = .2/N_ROWS
fig = make_subplots(rows=N_ROWS, cols=N_COLS, 
                    row_heights=[1,4,3], column_widths=[1,1],
                    horizontal_spacing=H_SPACING, vertical_spacing=V_SPACING,
                    row_titles=units, x_title='Total EPA')

fig.for_each_annotation(lambda a: a.update(x=-.135, textangle=-90, font=dict(weight='bold', size=14)) if a.text in units else())
fig.for_each_annotation(lambda a: a.update(y=.015, font=dict(weight='bold', size=14)) if a.text == 'Total EPA' else())

# Add bars
i = 0
for row in range(1, N_ROWS+1):
    for col in range(1, N_COLS+1):
        bar = bar_charts[i]
        
        for trace in bar.data:
            fig.add_trace(trace, row=row, col=col)

        i += 1

# Wordmarks
for i in range(len(wordmarks)):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))

    col_wid = (1 / N_COLS) - (H_SPACING / 2)
    fig.add_layout_image(
        x=(col_wid / 2) if i == 0 else 1 - (col_wid / 2),
        y=1.05,
        sizex=.2,
        sizey=.2,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )

# Format
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=2, color='#323232')))
fig.update_yaxes(
    autorange='reversed',
    linecolor='#f0f0f0', mirror=True

)
for i in range(N_ROWS):
    r_val = x_ranges[i][1] - x_ranges[i][0]
    fig.update_xaxes(
        range=x_ranges[i], 
        dtick=r_val // 7, 
        linecolor='#f0f0f0', 
        mirror=True,
        row=i+1
    )


# Credits
fig.add_annotation(
    text=f'Sorted by number of plays (bar text)<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='right'
)

fig.update_layout(
    template='nfl_template',
    title=dict(text=f'<b>EPA Box Score</b><br><sup>Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>'),
    height=700, width=700,
    margin=dict(t=100, r=15, b=65, l=90)
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'EPA Box Score - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png', scale=6, width=700, height=700)

## EPA

In [ ]:
''' EPA by Play Type '''

epa_summary = game_data.groupby(['posteam', 'play_type']).aggregate({

    'posteam': 'size', 
    'ep': 'sum',
    'epa': 'sum'
}).rename(columns={'posteam': 'Plays'})
epa_summary['EPA / Play'] = epa_summary['epa'] / epa_summary['Plays']

print(epa_summary.groupby('posteam')['epa'].sum())
print(epa_summary.to_string())

In [ ]:
''' Offensive Drives EPA '''

epa_summary = game_data.loc[(~game_data['Is Special Teams Play']), :].groupby(['game_half', 'drive', 'posteam']).aggregate(
    Plays=('posteam', lambda x: x[game_data['penalty'] == 0].shape[0]),
    StartEP=('ep', 'first'),
    EPA=('epa', 'sum'),
)
epa_summary['Drive End EP'] = epa_summary['StartEP'] + epa_summary['EPA']
epa_summary['Drive Exp. Pts'] = np.where(epa_summary['Drive End EP'] < 0, 0, epa_summary['Drive End EP'])
epa_summary['EPA / Play'] = epa_summary['EPA'] / epa_summary['Plays']

print(epa_summary.to_string())
print(epa_summary.groupby('posteam')['Drive Exp. Pts'].sum())

In [ ]:
print(game_data.loc[game_data['drive'] == 1].to_string())

In [ ]:
''' Best and Worst EPA Plays '''


for side in ['away', 'home']:
    col = 'home_team' if side == 'home' else 'away_team'
    team = game_data.loc[game_data.first_valid_index(), col]
    
    plays = game_data.loc[game_data['posteam_type'] == side, ['epa', 'qtr', 'time', 'yrdln', 'down', 'ydstogo', 'desc']].sort_values(by='epa', ascending=False)

    print(f'{team}')
    print('Best plays:')
    print(plays.head().to_string(index=False))
    print('Worst plays:')
    print(plays.tail().to_string(index=False))
